#### Prep Data

In [1]:
# set wd
import os
project_root = '/Users/kgedney/Documents/georgetown/anly503/project/'
os.chdir(project_root)

In [52]:
# prep
import pandas as pd
from pandas.api.types import CategoricalDtype

import bokeh

from bokeh.io import output_notebook
from bokeh.models.callbacks import CustomJS
from bokeh.models import ColumnDataSource, HoverTool, PrintfTickFormatter

from bokeh.layouts import widgetbox, column
from bokeh.models.widgets import RadioButtonGroup, Slider

from bokeh.plotting import figure, output_file, show

output_notebook()

Loading BokehJS ...

In [7]:
# load data
gdp           = pd.read_csv('data/gdp.csv')
log_gdp       = pd.read_csv('data/log_gdp.csv')
gdp_growth    = pd.read_csv('data/gdp_growth.csv')
df_electric   = pd.read_csv('data/electric.csv')
df_internet   = pd.read_csv('data/internet.csv')
df_cellphones = pd.read_csv('data/cellphones.csv')

# # add cat types back
# cat_type = CategoricalDtype(categories=['High income', 'Upper middle income', 
#                                         'Lower middle income', 'Low income'], 
#                             ordered=True)
# for df in [gdp, df_electric, df_internet, df_cellphones, log_gdp, gdp_growth]:
#     df['income_level'] = df['income_level'].astype(cat_type)

In [8]:
# add gdp growth levels in 2017 to each df
gdp_growth = gdp_growth.rename(columns={'2017': 'gdp_growth_2017'})

drop       = ['Eritrea', 'Somalia', 'South Sudan'] # drop countries with missing data
gdp_growth = gdp_growth[~gdp_growth.country.isin(drop)]

df_electric   = pd.merge(df_electric,   gdp_growth[['country', 'gdp_growth_2017']], on='country', how='inner')
df_internet   = pd.merge(df_internet,   gdp_growth[['country', 'gdp_growth_2017']], on='country', how='inner')
df_cellphones = pd.merge(df_cellphones, gdp_growth[['country', 'gdp_growth_2017']], on='country', how='inner')

#### Plot 
good ref: https://github.com/bokeh/bokeh/blob/master/examples/app/weather/main.py

In [6]:
# # add radio button (maybe these can be panes instead)
# radio_button_group = RadioButtonGroup(
#         labels=["Option 1", "Option 2", "Option 3"], active=0)

In [11]:
# helper function
def safe_cds(x):
    x = ColumnDataSource(x)
    for k in x.data.keys():
        x.data[k] = list(x.data[k])
    return x

def make_dataset(df, year_selected):
    df['x_axis_to_plot'] = df[str(year_selected)] # copy year_selected into special display column
    df['y_axis_to_plot'] = df['gdp_growth_2017']
    return safe_cds(df)

In [39]:
def make_widgets(source):

    #- create the interactive widgets
    year_select = Slider(start=1990, end=2016, value=1990, step=1, title="Choose a Year")

    #- define update functions: this is written in python but will be read with Javascript to update plots in html
    #- remember: must pass all vars that are updated as parameters
    def update_year(year_select=year_select, source=source):
        year_selected = year_select.value     # get value of active slider value
        new_data = source.data[year_selected] # get corresponding data from CDS for that year
        source.data['x_axis_to_plot'] = [n for n in new_data] # copy data into special column that will be displayd
        source.change.emit()  # refresh plot        
    
    #- set callbacks
    year_select.callback = CustomJS.from_py_func(update_year)
    
    return year_select

In [53]:
def make_plot(source):

    # set figure
    p = figure(title='GDP Growth and Electricty Access', 
               plot_width=750, plot_height=500, 
               x_axis_label = 'Electricty Access (% of Pop)', 
               y_axis_label ='GDP Growth (2017)',
               toolbar_location="above", 
               tools = "pan, box_zoom, reset")
    
    # format axis
    p.yaxis.formatter = PrintfTickFormatter(format='%0.0f %%')
    p.xaxis.formatter = PrintfTickFormatter(format='%0.0f %%')
    
    # add widgets
    year_select = make_widgets(source)

    # add glyphs
    p.circle(x      = 'x_axis_to_plot', 
             y      = 'y_axis_to_plot',
             source = source)
    
    # add hover
    TOOLTIPS = [
        ('Country', '@country'),
        ('Income Level', '@income_level'),
        ('GDP Growth Rate (2017)', '@gdp_growth_2017')]
    
    p.add_tools(HoverTool(tooltips=TOOLTIPS, toggleable=False))

    layout = column(p, widgetbox(year_select, width=750))
    
    show(layout)

In [54]:
# run plot
source = make_dataset(df_electric, year_selected=1990)
make_plot(source)

#### Plot 2